In [26]:
import torch

frame_diff = torch.rand(8, 6, 2, 2)
frame_diff[:, -4:] = 0

# print(frame_diff)

zero_diff = torch.sum(frame_diff == 0.0, dim=(2, 3))
# print(zero_diff)

B, T, H, W = frame_diff.shape
for i in range(B):
    matching_mask = (zero_diff[i] == H * W)
    matching_indices = torch.nonzero(matching_mask).flatten()
    for j in matching_indices:
        if j > 0:
            frame_diff[i, j] = frame_diff[i, j - 1]

print(frame_diff)

tensor([[[[0.5156, 0.9380],
          [0.2992, 0.4276]],

         [[0.5195, 0.0078],
          [0.2015, 0.7975]],

         [[0.5195, 0.0078],
          [0.2015, 0.7975]],

         [[0.5195, 0.0078],
          [0.2015, 0.7975]],

         [[0.5195, 0.0078],
          [0.2015, 0.7975]],

         [[0.5195, 0.0078],
          [0.2015, 0.7975]]],


        [[[0.4275, 0.0850],
          [0.2917, 0.6724]],

         [[0.6563, 0.4740],
          [0.3515, 0.2606]],

         [[0.6563, 0.4740],
          [0.3515, 0.2606]],

         [[0.6563, 0.4740],
          [0.3515, 0.2606]],

         [[0.6563, 0.4740],
          [0.3515, 0.2606]],

         [[0.6563, 0.4740],
          [0.3515, 0.2606]]],


        [[[0.7610, 0.6424],
          [0.5726, 0.8674]],

         [[0.4713, 0.3859],
          [0.9365, 0.9999]],

         [[0.4713, 0.3859],
          [0.9365, 0.9999]],

         [[0.4713, 0.3859],
          [0.9365, 0.9999]],

         [[0.4713, 0.3859],
          [0.9365, 0.9999]],

         [